In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predictive-maintainence/data.csv


In [3]:


# 1. Data Loading & Preprocessing
df=pd.read_csv("/kaggle/input/predictive-maintainence/data.csv")
from sklearn.model_selection import train_test_split

# Drop unneeded columns
drop_cols = ['id', 'acmotor']  # acmotor is always "stable", id is just identifier
df_clean = df.drop(columns=drop_cols).copy()

# Convert indicative columns to binary for averaging
failure_cols = ['bearings', 'wpump', 'radiator', 'exvalve']
df_clean[failure_cols] = df_clean[failure_cols].replace({
    "Ok": 0, "Noisy": 1,
    "Clean": 0, "Dirty": 1
})

# Split into train and test
train_df, test_df = train_test_split(df_clean, test_size=0.3, random_state=42, shuffle=True)


/tmp/ipykernel_10/1040889704.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean[failure_cols] = df_clean[failure_cols].replace({


In [4]:
def compute_risk(row):
    avg = row[failure_cols].mean()
    if avg == 0:
        return 0
    elif avg <= 0.5:
        return 1
    elif avg <=0.75:
        return 2
    else:
        return 3

train_df["risk"] = train_df.apply(compute_risk, axis=1)


In [5]:
# Drop failure columns + acmotor + id
X_train = train_df.drop(columns=failure_cols + ['risk'])
y_train = train_df['risk']


In [6]:
%pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier

# Initialize XGBoost for multi-class classification
xgb_model = XGBClassifier(
    objective='multi:softmax',  # softmax returns class labels directly
    num_class=4,                # 3 risk classes: 0, 1, 2
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 1.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/usr/local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [17:04:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=4, ...)

In [7]:
test_df["risk"] = test_df.apply(compute_risk, axis=1)

X_test = test_df.drop(columns=failure_cols + ['risk'])
y_test = test_df['risk']


In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Predict
y_pred = xgb_model.predict(X_test)

# Define all possible labels
labels = [0, 1, 2, 3]
target_names = ["No risk", "Low Risk", "Moderate Risk", "High Risk"]

# Evaluation
print(" Accuracy:", accuracy_score(y_test, y_pred))
print("\n Classification Report:")
print(classification_report(y_test, y_pred, labels=labels, target_names=target_names, zero_division=0))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred, labels=labels))


 Accuracy: 0.9933333333333333

 Classification Report:
               precision    recall  f1-score   support

      No risk       1.00      0.97      0.98        60
     Low Risk       0.99      1.00      1.00       240
Moderate Risk       0.00      0.00      0.00         0
    High Risk       0.00      0.00      0.00         0

     accuracy                           0.99       300
    macro avg       0.50      0.49      0.49       300
 weighted avg       0.99      0.99      0.99       300

Confusion Matrix:
[[ 58   2   0   0]
 [  0 240   0   0]
 [  0   0   0   0]
 [  0   0   0   0]]
